In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

from datasets import *
from models import *
from util import *

## Load data

In [2]:
p_pheno = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/pheno_01-12-21.csv'
p_conn = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/connectomes_01-12-21.csv'

data1 = caseControlDataset('SZ',p_pheno,p_conn)
data2 = caseControlDataset('BIP',p_pheno,p_conn)

/home/harveyaa/miniconda3/envs/MTL/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/home/harveyaa/miniconda3/envs/MTL/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types.Specify dtype option on import or set low_memory=False.
  """


In [3]:
train_data1, test_data1 = split_data(data1)
train_data2, test_data2 = split_data(data2)

trainloader1 = DataLoader(train_data1, batch_size=16, shuffle=True)
testloader1 = DataLoader(test_data1, batch_size=16, shuffle=True)

trainloader2 = DataLoader(train_data2, batch_size=16, shuffle=True)
testloader2 = DataLoader(test_data2, batch_size=16, shuffle=True)

In [4]:
model = HPSModel(encoder().double(),{'task1':head().double(),'task2':head().double()})

In [5]:
trainloaders = {'task1':trainloader1,'task2':trainloader2}
testloaders = {'task1':testloader1, 'task2':testloader2}

In [6]:
loss_fns = {'task1':nn.CrossEntropyLoss(),'task2':nn.CrossEntropyLoss()}
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

device = 'cpu'

In [7]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainMTL(trainloaders, model, loss_fns, optimizer)
    testMTL(testloaders, model, loss_fns)
print("Done!")

Epoch 1
-------------------------------
task1
Test Error: 
 Accuracy: 56.2%, Avg loss: 0.043255 

task2
Test Error: 
 Accuracy: 71.9%, Avg loss: 0.042067 

Epoch 2
-------------------------------
task1
Test Error: 
 Accuracy: 46.9%, Avg loss: 0.044657 

task2
Test Error: 
 Accuracy: 71.9%, Avg loss: 0.036892 

Epoch 3
-------------------------------
task1
Test Error: 
 Accuracy: 44.5%, Avg loss: 0.045507 

task2
Test Error: 
 Accuracy: 68.8%, Avg loss: 0.034846 

Epoch 4
-------------------------------
task1
Test Error: 
 Accuracy: 41.4%, Avg loss: 0.048592 

task2
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.034184 

Epoch 5
-------------------------------
task1
Test Error: 
 Accuracy: 46.9%, Avg loss: 0.047222 

task2
Test Error: 
 Accuracy: 75.0%, Avg loss: 0.034164 

Epoch 6
-------------------------------
task1
Test Error: 
 Accuracy: 48.4%, Avg loss: 0.047310 

task2
Test Error: 
 Accuracy: 75.0%, Avg loss: 0.036084 

Epoch 7
-------------------------------
task1
Test Error: 
 Accu